# People Detection and Follow

In [1]:
import numpy as np
import cv2
# from imutils.object_detection import non_max_suppression

# Dependencies :
# pip install numpy
# pip install opencv-python
# pip install imutils

# flag to indicate if there was click on the screen
r_clicked = False
l_clicked = False
window_name = 'People detection ( left click : redetection , right click : take screenshot )'


ped_cascade = cv2.CascadeClassifier('pedestrian.xml')


# Image Editing Help Methods

def detect_people(img):
    orig = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)

    # detect people in the image
    rects = ped_cascade.detectMultiScale(orig, 1.1, 2)
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])

    return rects


def point_inside_rect(p, rect):
    return rect[0] <= p[0] <= rect[2] and rect[1] <= p[1] <= rect[3]


def get_point_rect_index(p, rects):
    for i in range(len(rects)):
        if point_inside_rect(p, rects[i]):
            return i
    return -1


# events

def screen_click(event, x, y, flags, param):
    global l_clicked
    global r_clicked
    if event == cv2.EVENT_LBUTTONUP:
        l_clicked = True
    if event == cv2.EVENT_RBUTTONUP:
        r_clicked = True


# set event to windows click
cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, screen_click)


def main():
    global l_clicked
    global r_clicked
    write_output_video = True # set this to True if you want to record and save video

    # Consts

    video = 'v1.mp4'
    # params for ShiTomasi corner detection
    feature_params = dict(maxCorners=9999,
                          qualityLevel=0.1,
                          minDistance=16,
                          blockSize=11)

    # Parameters for lucas kanade optical flow
    lk_params = dict(winSize=(30, 30),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.01))

    colors = np.random.randint(0, 255, (400, 3))
    frames_limit = 1 * 24 # number of frames between the redetection
    follow_disappear_rate = 0.95 # the rate of mask disappear
    point_limit_ratio = 0.9 # the low threshold that indicate to redetect feature points ( like we did in class when there are not enough points )
    frame_counter_text_color = (255, 255, 255)
    frame_counter_text_pos = (6, 26)
    frame_counter_text_size = 1.75
    screenshot_pos = (6, 46)
    screeshot_text_size = 1.45
    screenshot_text_color = (0, 0, 255)
    screenshot_text_string = 'screen shot has been saved'

    
    # start of video capture
    cap = cv2.VideoCapture(video)
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    out = None
    
    # video record parameters
    if write_output_video:
        video_attr = ('out', 'mp4v', 20.0, old_gray.shape[1], old_gray.shape[0], '.mp4')
        fourcc = cv2.VideoWriter_fourcc(*video_attr[1])
        video_name = '{0}_{1}_{2}_{3}_{4}_output{5}'.format(*video_attr)
        out = cv2.VideoWriter(video_name, fourcc, video_attr[2], (video_attr[3], video_attr[4]))

    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, **feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    # detect the rectangles of the people in the current frame
    all_people = detect_people(old_frame)
    point_limit = len(p0) * point_limit_ratio
    frame_counter = 0
    fp_counter = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # recalc point if there are too few
        if l_clicked:
            print('looking for people in frame', frame_counter)
        if frame_counter % frames_limit == 0 or len(all_people) < 2 or l_clicked:
            all_people = detect_people(frame)
            l_clicked = False
        if len(p0) < point_limit or l_clicked:
            p0 = np.concatenate((p0, cv2.goodFeaturesToTrack(frame_gray, 100, 0.01, 10, None, None, 9)))
            point_limit = len(p0) * point_limit_ratio
            l_clicked = False

            
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params, )
        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()

            rect_id = get_point_rect_index(good_new[i], all_people)
            if rect_id != -1:
                # draw each point with the color of the person that it belongs to ( by the rect_id )
                fp_counter += 1
                mask = cv2.line(mask, (a, b), (c, d), colors[rect_id].tolist(), 2)
                frame = cv2.circle(frame, (a, b), 1, colors[rect_id].tolist(), -1)
        # write the number of the frame
        frame = cv2.putText(frame, str(frame_counter), frame_counter_text_pos, cv2.FONT_HERSHEY_PLAIN,
                            frame_counter_text_size,
                            frame_counter_text_color)

        # console stats
        if frame_counter % frames_limit == 0:
            print(np.round(fp_counter / frames_limit, 2),
                  'feature points in frame ( in average of', frames_limit, 'frames )')
            fp_counter = 0

        # people trace followup
        mask = (mask * follow_disappear_rate).astype(np.uint8)

        # people counter
        cv2.putText(frame, 'people : {0}'.format(len(all_people)), (6, frame.shape[0] - 20),
                    cv2.FONT_HERSHEY_PLAIN, 1.75, (0, 0, 0))
        img = cv2.add(frame, mask)
        # for rect in all_people:

        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

        # for (a, b, c, d) in all_people:
        # img = cv2.rectangle(img, (a, b), (c, d), (0, 255, 210), 4)

        # screen show is taken on right mouse click
        if r_clicked:
            print('screenshot of frame', frame_counter, 'has been saved')
            cv2.imwrite('screenshot_{0}.jpg'.format(frame_counter), img)
            r_clicked = False
            cv2.putText(img, screenshot_text_string, screenshot_pos, cv2.FONT_HERSHEY_PLAIN,
                        screeshot_text_size,
                        screenshot_text_color, 2)

        cv2.imshow(window_name, img)
        
        # add frame to output video stream
        if write_output_video:
            out.write(img)
            
        # exit if esc was pressed
        k = cv2.waitKey(30) & 0xff

        if k == 27:
            break
        frame_counter += 1

    cv2.destroyAllWindows()
    cap.release()
    if write_output_video:
        out.release()


if __name__ == '__main__':
    main()


1.46 feature points in frame ( in average of 24 frames )
32.46 feature points in frame ( in average of 24 frames )
34.42 feature points in frame ( in average of 24 frames )
42.71 feature points in frame ( in average of 24 frames )
33.17 feature points in frame ( in average of 24 frames )
29.67 feature points in frame ( in average of 24 frames )
84.92 feature points in frame ( in average of 24 frames )
99.08 feature points in frame ( in average of 24 frames )
105.58 feature points in frame ( in average of 24 frames )
114.17 feature points in frame ( in average of 24 frames )
107.5 feature points in frame ( in average of 24 frames )
124.17 feature points in frame ( in average of 24 frames )
102.54 feature points in frame ( in average of 24 frames )
128.62 feature points in frame ( in average of 24 frames )
157.25 feature points in frame ( in average of 24 frames )
